In [ ]:
from langchain_openai import AzureChatOpenAI
from langchain.tools import tool
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent
from langgraph.graph import StateGraph, START, END, Graph 
from models.database import *
from models.models import *
import langgraph
import os
from dotenv import load_dotenv

In [ ]:
load_dotenv()
MODELO = os.getenv("MODELO")
KEY = os.getenv("KEY")
VERSION = os.getenv("VERSION")
ENDPOINT = os.getenv("ENDPOINT")

In [6]:
llm = AzureChatOpenAI(
    api_key = KEY,
    api_version = VERSION,
    azure_endpoint = ENDPOINT,
    azure_deployment = MODELO
    )

In [ ]:
import openai
openai_client = openai.AzureOpenAI(
    api_key = KEY,
    api_version = VERSION,
    azure_endpoint = ENDPOINT
    )

In [ ]:
messages=[{"role": "system", "content": "Eres un asistente útil."},
        {"role": "user", "content": "¿Cuál es la capital de España?"}]

response = openai_client.chat.completions.create(
    model = MODELO,
    messages = messages,
    max_tokens=100
    )
response.choices[0].message.content

    ("pendiente_condicionada", "flujo"),
    ("pendiente_no_condicionada", "flujo"),
    ("grabar_cesta_srm", "flujo"),
    ("solicitar_firma_inf_prov_cond", "flujo"),
    ("firma_inf_prov_cond", "flujo"),
    ("lanzar_cesta_srm", "flujo"),
    ("aprobar_compra_manager", "aprobacion"),
    ("aprobar_compra_gestion", "aprobacion"),
    ("revision_compra", "flujo"),
    ("solicitar_oferta_provedoores", "flujo"),
    ("proveedor_envia_ofertn", "flujo"),
    ("consulta_acep_oferta", "flujo"),
    ("negociacion_proveedores", "flujo"),
    ("proponer_proveedor", "flujo"),
    ("aprobar_adjuducacion", "aprobacion"),
    ("facturar", "flujo"),
    ("fin", "flujo"),

In [7]:
# Agrego un proveedor de prueba
from sqlalchemy.orm import Session
from models.database import engine
from models.models import Proveedor

def agregar_proveedor_prueba():
    """
    Agrega un proveedor de prueba a la base de datos.
    """
    session = Session(bind=engine)

    # Crear un proveedor de prueba
    nuevo_proveedor = Proveedor(
        nombre="Apple",
        contacto="Soporte Apple",
        email="apple@proveedor.com",
        estado="aprobado"
    )
    session.add(nuevo_proveedor)
    session.commit()

    print(f"✅ Proveedor de prueba añadido: {nuevo_proveedor.nombre} (ID: {nuevo_proveedor.id_proveedor})")
    session.close()

# Ejecutar la función para agregar el proveedor
agregar_proveedor_prueba()


2025-03-15 19:22:03,362 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-15 19:22:03,366 INFO sqlalchemy.engine.Engine INSERT INTO proveedores (nombre, contacto, email, estado, fecha_registro) VALUES (?, ?, ?, ?, ?)
2025-03-15 19:22:03,367 INFO sqlalchemy.engine.Engine [generated in 0.00110s] ('Apple', 'Soporte Apple', 'apple@proveedor.com', 'aprobado', '2025-03-15 18:22:03.366162')
2025-03-15 19:22:03,370 INFO sqlalchemy.engine.Engine COMMIT
2025-03-15 19:22:03,379 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-15 19:22:03,382 INFO sqlalchemy.engine.Engine SELECT proveedores.id_proveedor AS proveedores_id_proveedor, proveedores.nombre AS proveedores_nombre, proveedores.contacto AS proveedores_contacto, proveedores.email AS proveedores_email, proveedores.estado AS proveedores_estado, proveedores.fecha_registro AS proveedores_fecha_registro 
FROM proveedores 
WHERE proveedores.id_proveedor = ?
2025-03-15 19:22:03,384 INFO sqlalchemy.engine.Engine [generated in 0.00116s] (1

In [18]:
def obtener_datos_pedido(entrada_usuario):
    ''' Nodo que usa IA para obtener los datos del pedido del usuario '''
    # Se crea la sesión de SQLAlchemy	
    session = Session(bind=engine)

    prompt = f''' 
    Eres el encargado de obtener la información de un pedido.
    Se necesita conocer:
    - Nombre del proveedor
    - Moneda en la que se realizó la compra
    - Presupuesto estimado

    {entrada_usuario}

    Devuevle los datos en un formato claro como:
    "Proveedor: Proveedor X, Moneda: EUR, Presupuesto: 5000"
    '''
    response = llm.invoke(prompt)
    datos_ia = response.content.strip()

    print(f'IA: {datos_ia}')

    # Extraer los datos del texto
    datos_extraidos = {}
    for item in datos_ia.split(','):
        key, value = item.strip().split(':')
        datos_extraidos[key.lower().strip()] = value.strip()
    
    nombre_proveedor = datos_extraidos.get('proveedor')
    codigo_moneda = datos_extraidos.get('moneda')
    presupuesto = float(datos_extraidos.get('presupuesto', 1000)) # Por defecto 1000

    print(f"📦 Datos extraídos: Proveedor: {nombre_proveedor}, Moneda: {codigo_moneda}, Presupuesto: {presupuesto}")
    
    # Verificar que la moneda existe en la BD
    moneda = session.query(Moneda).filter(Moneda.codigo == codigo_moneda).first()
    if not moneda:
        print(f"⚠️ Moneda '{codigo_moneda}' no encontrada. Se usará EUR por defecto.")
        moneda = session.query(Moneda).filter_by(codigo="EUR").first()

    # Verificar si el proveedor ya existe en la BD
    proveedor = session.query(Proveedor).filter_by(nombre=nombre_proveedor).first()

    if proveedor:
        print(f"✅ Proveedor '{proveedor.nombre}' encontrado en la base de datos.")
        es_nuevo = False
    else:
        print(f"⚠️ Proveedor '{nombre_proveedor}' no encontrado. Requiere validación.")
        es_nuevo = True
    session.close()

    return {
        "id_proveedor": proveedor.id_proveedor if proveedor else None,
        "nombre_proveedor": nombre_proveedor,
        "id_usuario": 1,
        "nombre_cesta": "Cesta generada automáticamente",
        "tipo_compra": "ordinaria",
        "presupuesto": presupuesto,
        "id_moneda": moneda.id_moneda,
        "descripcion_pedido": "Pedido generado automáticamente",
        "pedido_tipoimp": "K",
        "es_nuevo": es_nuevo  # 🔹 Indica si el proveedor debe validarse antes de continuar
    }


In [19]:
consulta_usuario = """
Necesito comprar 15 monitores y 10 teclados mecánicos para la oficina. 
El presupuesto estimado es de 7000 dólares. 
Quiero hacer la compra a Dell, pero si no es posible, podemos buscar otro proveedor.
"""

# Probar obtener_datos_pedido con una consulta de usuario
resultado = obtener_datos_pedido(consulta_usuario)

# Mostrar los resultados obtenidos
print("\n🔹 RESULTADO DE LA FUNCIÓN 🔹")
for clave, valor in resultado.items():
    print(f"{clave}: {valor}")

# Verificar si el proveedor fue detectado correctamente
if resultado["es_nuevo"]:
    print("\n⚠️ El proveedor es nuevo y requiere validación.")
else:
    print("\n✅ El proveedor ya existe en la base de datos y se usará directamente.")


IA: Proveedor: Dell, Moneda: USD, Presupuesto: 7000
📦 Datos extraídos: Proveedor: Dell, Moneda: USD, Presupuesto: 7000.0
2025-03-15 19:34:46,239 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-15 19:34:46,241 INFO sqlalchemy.engine.Engine SELECT monedas.id_moneda AS monedas_id_moneda, monedas.codigo AS monedas_codigo, monedas.nombre AS monedas_nombre 
FROM monedas 
WHERE monedas.codigo = ?
 LIMIT ? OFFSET ?
2025-03-15 19:34:46,241 INFO sqlalchemy.engine.Engine [generated in 0.00073s] ('USD', 1, 0)
2025-03-15 19:34:46,244 INFO sqlalchemy.engine.Engine SELECT proveedores.id_proveedor AS proveedores_id_proveedor, proveedores.nombre AS proveedores_nombre, proveedores.contacto AS proveedores_contacto, proveedores.email AS proveedores_email, proveedores.estado AS proveedores_estado, proveedores.fecha_registro AS proveedores_fecha_registro 
FROM proveedores 
WHERE proveedores.nombre = ?
 LIMIT ? OFFSET ?
2025-03-15 19:34:46,245 INFO sqlalchemy.engine.Engine [generated in 0.00129s] ('De

In [ ]:
def grabar_cesta_srm():
    tools = []
    agent_pedido = create_react_agent(llm, tools)


In [ ]:
def aprobacion_proveedor():
    tools = []
    agent_pedido = create_react_agent(llm, tools)

In [ ]:
# Codigo para generar el flujo

graph_builder = StateGraph(dict)
graph_builder = Graph()

# Nodos al grafo

graph_builder.add_node("obtener_datos_del_pedido", obtener_datos_pedido)
graph_builder.add_node("grabar_cesta_srm", grabar_cesta_srm)
graph_builder.add_node("aprobacion_proveedor", aprobacion_proveedor)

In [ ]:
# 📌 Conectar nodos condicionalmente
def decidir_siguiente_nodo(datos):
    if datos["es_nuevo"]:
        return "aprobacion_proveedor"
    return "crear_pedido"

In [ ]:
graph.add_conditional_edges("obtener_datos_pedido", decidir_siguiente_nodo)
graph.add_edge("aprobacion_proveedor", "crear_pedido")  # Si se aprueba, se crea el pedido
